In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:

import google.generativeai as genai

In [3]:
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [4]:
model=genai.GenerativeModel('gemini-pro')

print(model)

 genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
)


In [5]:

prompt_template = """
[Context]
As a Data Scientist expert in SQL and given a sales delta lake table that containts the daily sales of a sporting goods store.
You receive human written text requests to get information from the sales table. You need to generate the SQL statement to get the information requested.
The table has the following schema:
- sale_id
- product_name
- seller
- quantity_sold
- sale_date
- customer_name
- total_amount (in usd)
The path of the delta lake table is: "s3://bucket/sales".

When generating the query take into account the following:
1. The select statement follow the following format: Select * from delta.`s3://bucket/sales`
2. If you need to use a where clause to get the sales date minus some days, use the following format : sale_date >= date_sub(current_date(), 7)

Answer 'NoAnswer' if you do not know the answer.
[Request]
{request}
"""

model_response = model.generate_content(
    prompt_template.format(request="Get me the top ten sellers and the sells in USD in the past 3 days"),
    generation_config={"temperature": 0}
)

print(model_response.text)

```sql
SELECT seller, SUM(total_amount) AS total_sales_usd
FROM delta.`s3://bucket/sales`
WHERE sale_date >= date_sub(current_date(), 3)
GROUP BY seller
ORDER BY total_sales_usd DESC
LIMIT 10;
```


In [43]:
sales_table_path = "s3://{aws_access}:{aws_secret}@mydev-databricks/deltalake/sales".format(
    aws_access=os.environ['AWS_ACCESS_KEY'], aws_secret=os.environ['AWS_SECRET_KEY']
)
sql_statement = model_response.text.split('\n')[1:-1]
sql_statement = '\n'.join(sql_statement)

print(sql_statement)

SELECT seller, SUM(total_amount) AS total_sales_usd
FROM delta.`s3://bucket/sales`
WHERE sale_date >= date_sub(current_date(), 3)
GROUP BY seller
ORDER BY total_sales_usd DESC
LIMIT 10;


In [44]:
sql_statement = sql_statement.replace('s3://bucket/sales', sales_table_path)

In [45]:
import requests
import json

url = "{databricks_api}/sql/statements/".format(databricks_api=os.environ['DATABRICKS_API'])

payload = json.dumps({
  "warehouse_id": os.environ['DATABRICKS_WAREHOUSE_ID'],
  "statement": sql_statement,
  "wait_timeout": "50s"
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer {databricks_token}'.format(databricks_token=os.environ['DATABRICKS_TOKEN'])
}


In [47]:
response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"statement_id":"01eec1d1-06f2-1fdd-9026-25a00f8aaf1e","status":{"state":"SUCCEEDED"},"manifest":{"format":"JSON_ARRAY","schema":{"column_count":2,"columns":[{"name":"seller","type_text":"STRING","type_name":"STRING","position":0},{"name":"total_sales_usd","type_text":"DOUBLE","type_name":"DOUBLE","position":1}]},"total_chunk_count":1,"chunks":[{"chunk_index":0,"row_offset":0,"row_count":10}],"total_row_count":10,"truncated":false},"result":{"chunk_index":0,"row_offset":0,"row_count":10,"data_array":[["David Moore","1180.0"],["Michael Williams","1100.0"],["Emma Jones","920.0"],["Sophia Wilson","880.0"],["Jane Johnson","860.0"],["Ava Taylor","700.0"],["Olivia Davis","700.0"],["William Miller","680.0"],["John Smith","680.0"],["Robert Brown","660.0"]]}}


In [48]:
response_json = json.loads(response.text)
print(response_json)

{'statement_id': '01eec1d1-06f2-1fdd-9026-25a00f8aaf1e', 'status': {'state': 'SUCCEEDED'}, 'manifest': {'format': 'JSON_ARRAY', 'schema': {'column_count': 2, 'columns': [{'name': 'seller', 'type_text': 'STRING', 'type_name': 'STRING', 'position': 0}, {'name': 'total_sales_usd', 'type_text': 'DOUBLE', 'type_name': 'DOUBLE', 'position': 1}]}, 'total_chunk_count': 1, 'chunks': [{'chunk_index': 0, 'row_offset': 0, 'row_count': 10}], 'total_row_count': 10, 'truncated': False}, 'result': {'chunk_index': 0, 'row_offset': 0, 'row_count': 10, 'data_array': [['David Moore', '1180.0'], ['Michael Williams', '1100.0'], ['Emma Jones', '920.0'], ['Sophia Wilson', '880.0'], ['Jane Johnson', '860.0'], ['Ava Taylor', '700.0'], ['Olivia Davis', '700.0'], ['William Miller', '680.0'], ['John Smith', '680.0'], ['Robert Brown', '660.0']]}}


In [50]:
columns = response_json['manifest']['schema']['columns']
print(columns)

[{'name': 'seller', 'type_text': 'STRING', 'type_name': 'STRING', 'position': 0}, {'name': 'total_sales_usd', 'type_text': 'DOUBLE', 'type_name': 'DOUBLE', 'position': 1}]


In [51]:
data = response_json['result']['data_array']
print(data)

[['David Moore', '1180.0'], ['Michael Williams', '1100.0'], ['Emma Jones', '920.0'], ['Sophia Wilson', '880.0'], ['Jane Johnson', '860.0'], ['Ava Taylor', '700.0'], ['Olivia Davis', '700.0'], ['William Miller', '680.0'], ['John Smith', '680.0'], ['Robert Brown', '660.0']]


In [55]:
import pandas as pd

In [56]:
# Extract column names from JSON
column_names = [column['name'] for column in columns]

# Create Pandas DataFrame
df = pd.DataFrame(data, columns=column_names)

In [60]:
from IPython.display import display, HTML
display(df)

,seller,total_sales_usd
0,David Moore,1180.0
1,Michael Williams,1100.0
2,Emma Jones,920.0
3,Sophia Wilson,880.0
4,Jane Johnson,860.0
5,Ava Taylor,700.0
6,Olivia Davis,700.0
7,William Miller,680.0
8,John Smith,680.0
9,Robert Brown,660.0
